In [1]:
import sys
sys.path.insert(1, '../..')

import torch
import torch.nn as nn
import random
import pandas as pd
import numpy as np

from library.evaluation import ConfusionMatrix

random.seed(33)

dataset_name = "Twitter15_CrossVal"
unique_name = "ByT5"

In [2]:
vectors = np.loadtxt("../../data/processed/vectors/Twitter15_ByT5_vectors.txt", delimiter=",")
vectors.shape

(1490, 1536)

In [3]:
data = pd.read_csv("../../data/processed/twitter15_dataset_with_tvt.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,tvt,cv_fold,tt
0,731166399389962242,🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...,unverified,training,1,training
1,714598641827246081,an open letter to trump voters from his top st...,unverified,training,1,test
2,691809004356501505,america is a nation of second chances —@potus ...,non-rumor,training,2,training
3,693204708933160960,"brandon marshall visits and offers advice, sup...",non-rumor,training,1,training
4,551099691702956032,rip elly may clampett: so sad to learn #beverl...,true,training,3,training


In [4]:
labels_str = data['label'].unique().tolist()
labels_str

['unverified', 'non-rumor', 'true', 'false']

In [5]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d['label'])
#     labels.append([1 if j == lab else 0 for j in range(len(labels_str))])
    labels.append(lab)
labels[:10]

[0, 0, 1, 1, 2, 1, 0, 2, 0, 3]

In [6]:
test_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])
test_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])

In [7]:
print(test_vectors.shape)
print(test_labels.shape)

(151, 1536)
(151,)


In [8]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import os
from typing import Callable


class NNModel(nn.Module):
    def __init__(
        self,
        n_input: int,
        n_output: int = 1
    ):
        super(NNModel, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(n_input, 512),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(512),
            nn.Dropout(p=0.5),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(512),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(256),
            nn.Dropout(p=0.5),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(128),
            nn.Dropout(p=0.5),
            nn.Linear(128, n_output)
        )

    def forward(self, input):
        return self.main(input)
    

class NNClassifier:
    def __init__(self,
        n_input: int,
        n_output: int = 1,
        criterion: Callable = nn.BCELoss,
        beta1: float = 0.5,
        lr: float = 0.0002,
        device: str = None
    ):
        super(NNClassifier, self).__init__()
        self.model = NNModel(n_input, n_output)

        self.criterion = criterion()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)

        if not device or device not in ['cpu', 'cuda']:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device

        self.model = self.model.to(self.device)
        if self.device == 'cuda':
            self.model = torch.nn.DataParallel(self.model)
            cudnn.benchmark = True
    
    def load_pretrained(self, filepath: str, key: str = "net", is_parallel: bool = False):
        checkpoint = torch.load(filepath)
        if is_parallel:
            self.model = torch.nn.DataParallel(self.model)
            cudnn.benchmark = True

        print(f"loading model from {filepath}...")
#         print(checkpoint[key])
        self.model.load_state_dict(checkpoint[key], strict=False)
    
    def save_model(self, saves: str):
        print(f"Saving model...")
        state = {
            'net': self.model.state_dict()
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f"../../data/models/{saves}.pth")
    
    def train_eval(self,
        train_x, train_y,
        test_x, test_y,
        n_iter: int = 100,
        batch_size: int = 128,
        saves: str = None
    ):
        trainset = torch.utils.data.TensorDataset(train_x, train_y) # create your datset
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size) # create your dataloader

        testset = torch.utils.data.TensorDataset(test_x, test_y) # create your datset
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size) # create your dataloader

        train_accs = []
        train_losses = []
        test_accs = []
        test_losses = []

        print(f"Using {self.device}")
        best_acc = 0
        current_loss = 1000
        best_test_acc = 0
        epoch = 0
        start_time = time.time()
        results = {}
        while True:
            epoch += 1
            self.model.train()
            train_loss = 0
            correct = 0
            total = 0
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                self.model.zero_grad()
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets.long())
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()
                total += targets.size(0)
#                 for i, output in enumerate(outputs.tolist()):
#                     if targets[i,0].tolist() == round(output[0]):
#                         correct += 1

#             train_acc = round(100*correct/total, 4)
#             train_accs.append(train_acc)
            train_losses.append(train_loss)

            self.model.eval()
            test_loss = 0
            test_acc = 0
            with torch.no_grad():
                inputs, targets = test_x.to(self.device), test_y.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets.long())

                test_loss += loss.item()
                
                preds = self.predict(test_x)
                conf_mat = ConfusionMatrix(
                    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_y]),
                    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds.cpu().numpy()]),
                    binary=False
                )
                conf_mat.evaluate(logs=False)
                test_acc = conf_mat.accuracy

            test_losses.append(test_loss)
            
#             if (epoch) % round(n_iter/20) == 0:
#                 print(f"-- Epoch {epoch}, Train Loss : {train_loss}, Test Loss : {test_loss}")

            # Save checkpoint.
#             if saves and test_loss < best_loss:
#                 print(f"Saving after new best loss : {test_loss}")
#                 best_loss = test_loss
            if saves and test_acc > best_test_acc:
#                 print(f"Saving after new best accuracy : {test_acc}")
                best_test_acc = test_acc

                state = {
                    'net': self.model.state_dict(),
                }
                if not os.path.isdir('models'):
                    os.mkdir('models')
                torch.save(state, f"../../data/models/{saves}.pth")
            
            if epoch >= n_iter:
                break

#         # visualizing accuracy over epoch
#         fig, ax2 = plt.subplots(1)
#         plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=1.5, wspace=0.4)

#         ax2.plot([i for i in range(len(train_losses))], train_losses, c='b', marker="o", label='Train Loss')
#         ax2.plot([i for i in range(len(test_losses))], test_losses, c='r', marker="o", label='Test Loss')
#         ax2.set_ylabel('Loss')
#         ax2.set_xlabel('Epoch')
#         ax2.set_xlim(0, len(train_losses))
#         ax2.set_ylim(min([min(train_losses), min(test_losses)])*0.1, max([max(train_losses), max(test_losses)]))
#         ax2.title.set_text(f"Loss over time (epoch)")
#         ax2.legend(loc='lower right')

#         plt.show()
    
    def predict(self, input_x):
        self.model.eval()
        with torch.no_grad():
            preds = self.model(torch.Tensor(input_x))
            preds = torch.log_softmax(preds, dim = 1)
            _, preds = torch.max(preds, dim = 1)
            return preds

In [9]:
print("3-Fold Cross Validation with 4-Layer Linear Network")

folds = [1, 2, 3]
for fold in folds:
    val_folds = [fold]
    train_folds = folds.copy()
    train_folds.remove(fold)

    train_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    train_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    print(f"Fold-{fold} Cross Validation")
    model_name = f"{dataset_name}_4LayerNet_L2Reg_{unique_name}_fold-{fold}"
    model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)
    model.train_eval(torch.Tensor(train_vectors),
                    torch.Tensor(train_labels),
                    torch.Tensor(val_vectors),
                    torch.Tensor(val_labels),
                    saves=model_name,
                    n_iter=1000,
                    batch_size=512)

    model.load_pretrained(f"../../data/models/{model_name}.pth")

    print(f"\n-------- Fold-{fold} Results --------")
    preds = model.predict(val_vectors)

    preds = preds.cpu().numpy()

    conf_mat = ConfusionMatrix(
        labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in val_labels]),
        predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
        binary=False,
        model_name=model_name
    )
    conf_mat.evaluate(classes=labels_str)
    print(f"-------- Fold-{fold} End --------\n")

3-Fold Cross Validation with 4-Layer Linear Network
Fold-1 Cross Validation
Using cuda
loading model from ../../data/models/Twitter15_CrossVal_4LayerNet_L2Reg_ByT5_fold-1.pth...

-------- Fold-1 Results --------
474 vs 474
Multi Class Evaluation

Class unverified Evaluation
- Precision : 53.226 %
- Recall : 55.932 %
- F1 : 0.54545

Class non-rumor Evaluation
- Precision : 58.333 %
- Recall : 57.273 %
- F1 : 0.57798

Class true Evaluation
- Precision : 74.016 %
- Recall : 75.806 %
- F1 : 0.749

Class false Evaluation
- Precision : 53.913 %
- Recall : 50.82 %
- F1 : 0.52321

Combined Evaluation
- Accuracy : 60.127 %
- Precision : 59.872 %
- Recall : 59.958 %
- F1 : 0.59915

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_4LayerNet_L2Reg_ByT5_fold-1, 60.127, 59.872, 59.958, 0.59915, 53.226, 55.932, 0.54545, 58.333, 57.273, 0.57798, 74.016, 75.806, 0.749, 53.913, 50.82, 0.52321, 
-------- Fold-1 End --------

Fold-2 Cross Valida

In [10]:
fold_n = 1
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_L2Reg_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-1
loading model from ../../data/models/Twitter15_CrossVal_4LayerNet_L2Reg_ByT5_fold-1.pth...

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 48.276 %
- Recall : 38.889 %
- F1 : 0.43077

Class non-rumor Evaluation
- Precision : 58.824 %
- Recall : 78.947 %
- F1 : 0.67416

Class true Evaluation
- Precision : 74.359 %
- Recall : 74.359 %
- F1 : 0.74359

Class false Evaluation
- Precision : 50.0 %
- Recall : 42.105 %
- F1 : 0.45714

Combined Evaluation
- Accuracy : 58.94 %
- Precision : 57.865 %
- Recall : 58.575 %
- F1 : 0.58218

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_4LayerNet_L2Reg_ByT5_fold-1, 58.94, 57.865, 58.575, 0.58218, 48.276, 38.889, 0.43077, 58.824, 78.947, 0.67416, 74.359, 74.359, 0.74359, 50.0, 42.105, 0.45714, 
-------- Testing End --------



In [11]:
fold_n = 2
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_L2Reg_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-2
loading model from ../../data/models/Twitter15_CrossVal_4LayerNet_L2Reg_ByT5_fold-2.pth...

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 60.0 %
- Recall : 58.333 %
- F1 : 0.59155

Class non-rumor Evaluation
- Precision : 68.889 %
- Recall : 81.579 %
- F1 : 0.74699

Class true Evaluation
- Precision : 68.293 %
- Recall : 71.795 %
- F1 : 0.7

Class false Evaluation
- Precision : 66.667 %
- Recall : 52.632 %
- F1 : 0.58824

Combined Evaluation
- Accuracy : 66.225 %
- Precision : 65.962 %
- Recall : 66.085 %
- F1 : 0.66023

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_4LayerNet_L2Reg_ByT5_fold-2, 66.225, 65.962, 66.085, 0.66023, 60.0, 58.333, 0.59155, 68.889, 81.579, 0.74699, 68.293, 71.795, 0.7, 66.667, 52.632, 0.58824, 
-------- Testing End --------



In [12]:
fold_n = 3
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_L2Reg_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-3
loading model from ../../data/models/Twitter15_CrossVal_4LayerNet_L2Reg_ByT5_fold-3.pth...

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 56.667 %
- Recall : 47.222 %
- F1 : 0.51515

Class non-rumor Evaluation
- Precision : 58.824 %
- Recall : 78.947 %
- F1 : 0.67416

Class true Evaluation
- Precision : 69.048 %
- Recall : 74.359 %
- F1 : 0.71605

Class false Evaluation
- Precision : 75.0 %
- Recall : 55.263 %
- F1 : 0.63636

Combined Evaluation
- Accuracy : 64.238 %
- Precision : 64.884 %
- Recall : 63.948 %
- F1 : 0.64413

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_4LayerNet_L2Reg_ByT5_fold-3, 64.238, 64.884, 63.948, 0.64413, 56.667, 47.222, 0.51515, 58.824, 78.947, 0.67416, 69.048, 74.359, 0.71605, 75.0, 55.263, 0.63636, 
-------- Testing End --------

